In [1]:
from precomputed_python import AnnotationReader, morton_code_to_gridpt, compressed_morton_code
import numpy as np
import pandas as pd
import asyncio
import tensorstore as ts

In [2]:
# pre_ann=AnnotationReader("gs://neuroglancer-20191211_fafbv14_buhmann2019_li20190805")
# ann = pre_ann.get_by_id(233446588)
pre_ann = AnnotationReader("gs://allen_neuroglancer_ccf/mesoscale_connectivity_atlas_points_v9")
# pre_ann = AnnotationReader("gs://allen_neuroglancer_ccf/merscope_ccffix_1")


I0331 16:39:22.524779 7262765 google_auth_provider.cc:149] Using credentials at /Users/forrestc/.config/gcloud/application_default_credentials.json
I0331 16:39:22.526231 7262765 google_auth_provider.cc:156] Using OAuth2 AuthProvider


In [ ]:
pre_ann.read_annotations_in_chunk('spatial0', chunk_index=[5,3,5])

,geometry,injection_volume,transgenic_line,structure_abbrev,structure_name,gender,structure_color,padding15
ID,,,,,,,,
183225830,"[6060.0, 4010.0, 5510.0]",0.236394,Grm2-Cre_MR90,PVT,Paraventricular nucleus of the thalamus,F,"[255, 144, 159]",[0]
125830911,"[5510.0, 3250.0, 5690.0]",0.268019,nan,LSr,"Lateral septal nucleus, rostral (rostroventral...",M,"[144, 203, 237]",[0]
182341627,"[5350.0, 4050.0, 6040.0]",0.041936,Syt6-Cre_KI148,TRS,Triangular nucleus of septum,F,"[150, 167, 211]",[0]
268399145,"[6040.0, 3580.0, 6040.0]",0.050636,Ppp1r17-Cre_NL146,MD,Mediodorsal nucleus of thalamus,F,"[255, 144, 159]",[0]
287950390,"[5510.0, 3560.0, 6040.0]",0.046494,Calb2-IRES-Cre,TRS,Triangular nucleus of septum,F,"[150, 167, 211]",[0]
178486024,"[5140.0, 3570.0, 6140.0]",0.040623,Cart-Tg1-Cre,LSr,"Lateral septal nucleus, rostral (rostroventral...",F,"[144, 203, 237]",[0]


In [24]:
ts_spec=pre_ann.spatial_ts_dict['spatial0'][0].spec().to_json()
ts_spec

{'base': {'bucket': 'allen_neuroglancer_ccf',
  'driver': 'gcs',
  'path': 'mesoscale_connectivity_atlas_points_v9/spatial0/'},
 'driver': 'neuroglancer_uint64_sharded',
 'metadata': {'@type': 'neuroglancer_uint64_sharded_v1',
  'data_encoding': 'gzip',
  'hash': 'murmurhash3_x86_128',
  'minishard_bits': 2,
  'minishard_index_encoding': 'gzip',
  'preshift_bits': 10,
  'shard_bits': 0}}

In [44]:
import os 
import tensorstore as ts
import numpy as np 

source = "gs://allen_neuroglancer_ccf/mesoscale_connectivity_atlas_points_v9"
info_ts = ts.open({'kvstore': os.path.join(source, 'info'), 'driver':'json'}).result()
info_d=info_ts.read().result().item()
spatial_md = info_d['spatial'][0]
spatial_ts = ts.KvStore.open({
  'driver':'neuroglancer_uint64_sharded',
  'base': os.path.join(source, spatial_md['key']),
  'metadata': spatial_md['sharding']
}).result()
grid_shape = np.array(spatial_md['grid_shape'], dtype=np.int32)
center_pt=grid_shape//2
mc = compressed_morton_code(center_pt, grid_shape)
print(mc)
key = np.ascontiguousarray(int(mc), dtype=">u8").tobytes()
print(key)
spatial_ts[key]

350
b'\x00\x00\x00\x00\x00\x00\x01^'


b'\x08\x00\x00\x00\x00\x00\x00\x00\x00\x80\xb6E\x00\xa0zE\x00\xd0\xd9E7*&>\x00\x00y\x00y\x00\x00\x00\xff\x90\x9f\x00\x00\xa0\xbeE\x00\xe0{E\x00P\xc8E\x0f2\xee=\x1e\x00y\x00y\x00\x01\x00\xff\x90\x9f\x00\x00\x80\xbbE\x00\xa0aE\x00\xc0\xc6E\x96N/>\x00\x00y\x00y\x00\x00\x00\xff\x90\x9f\x00\x00\xa0\xb9E\x00\xe0XE\x000\xd4E\xa3\x880=\x06\x00y\x00y\x00\x01\x00\xff\x90\x9f\x00\x00\xd0\xbbE\x00\xc0nE\x00\x90\xd8E\x06>\xd0=)\x00y\x00y\x00\x00\x00\xff\x90\x9f\x00\x00\x00\xa5E\x00\xe0bE\x00\xf0\xc8E\xabD\x01>G\x00g\x00g\x00\x01\x00\x90\xcb\xed\x00\x00`\xa4E\x00\xc0PE\x00P\xc3E\xddC7=x\x00\xca\x00\xca\x00\x01\x00\x90\xcb\xed\x00\x00\x00\xbeE\x00\x80mE\x00`\xc7E\x10@J;\x10\x00\xd7\x00\xd7\x00\x00\x00\xff\x90\x9f\x00\xe9\r\xf8\x05\x00\x00\x00\x00\xfewl\x11\x00\x00\x00\x00S\x05\xd2\x06\x00\x00\x00\x00\xdft\x14\x11\x00\x00\x00\x008\xdcn\x11\x00\x00\x00\x00\xe3z\xee\x11\x00\x00\x00\x00\xb5\xb6u\x1f\x00\x00\x00\x00\x12WH\t\x00\x00\x00\x00'

In [49]:
spatial_ts.spec().to_json()

{'base': {'bucket': 'allen_neuroglancer_ccf',
  'driver': 'gcs',
  'path': 'mesoscale_connectivity_atlas_points_v9/spatial0/'},
 'driver': 'neuroglancer_uint64_sharded',
 'metadata': {'@type': 'neuroglancer_uint64_sharded_v1',
  'data_encoding': 'gzip',
  'hash': 'murmurhash3_x86_128',
  'minishard_bits': 0,
  'minishard_index_encoding': 'gzip',
  'preshift_bits': 10,
  'shard_bits': 0}}

In [48]:
pre_ann.spatial_ts_dict['spatial0'][0].spec().to_json()

{'base': {'bucket': 'allen_neuroglancer_ccf',
  'driver': 'gcs',
  'path': 'mesoscale_connectivity_atlas_points_v9/spatial0/'},
 'driver': 'neuroglancer_uint64_sharded',
 'metadata': {'@type': 'neuroglancer_uint64_sharded_v1',
  'data_encoding': 'gzip',
  'hash': 'murmurhash3_x86_128',
  'minishard_bits': 2,
  'minishard_index_encoding': 'gzip',
  'preshift_bits': 10,
  'shard_bits': 0}}

In [18]:
ts_dv=ts.KvStore.open('gs://allen_neuroglancer_ccf/merscope_ccffix_1/spatial0/').result()

In [19]:
ts_dv.list().result()

[b'0_0_0',
 b'0_2_4',
 b'0_2_5',
 b'0_2_6',
 b'0_2_7',
 b'0_3_3',
 b'0_3_4',
 b'0_3_5',
 b'0_3_6',
 b'0_3_7',
 b'0_4_3',
 b'0_4_4',
 b'0_4_5',
 b'0_4_6',
 b'0_4_7',
 b'0_5_3',
 b'0_5_4',
 b'0_5_5',
 b'0_5_6',
 b'0_5_7',
 b'0_6_4',
 b'0_6_6',
 b'0_6_7',
 b'10_0_4',
 b'10_0_5',
 b'10_0_6',
 b'10_1_1',
 b'10_1_2',
 b'10_1_3',
 b'10_1_4',
 b'10_1_5',
 b'10_1_6',
 b'10_1_7',
 b'10_1_8',
 b'10_2_1',
 b'10_2_2',
 b'10_2_3',
 b'10_2_4',
 b'10_2_5',
 b'10_2_6',
 b'10_2_7',
 b'10_2_8',
 b'10_2_9',
 b'10_3_0',
 b'10_3_1',
 b'10_3_2',
 b'10_3_3',
 b'10_3_4',
 b'10_3_5',
 b'10_3_6',
 b'10_3_7',
 b'10_3_8',
 b'10_3_9',
 b'10_4_0',
 b'10_4_1',
 b'10_4_10',
 b'10_4_2',
 b'10_4_3',
 b'10_4_4',
 b'10_4_5',
 b'10_4_6',
 b'10_4_7',
 b'10_4_8',
 b'10_4_9',
 b'10_5_0',
 b'10_5_1',
 b'10_5_10',
 b'10_5_2',
 b'10_5_3',
 b'10_5_4',
 b'10_5_5',
 b'10_5_6',
 b'10_5_7',
 b'10_5_8',
 b'10_5_9',
 b'10_6_3',
 b'10_6_4',
 b'10_6_5',
 b'10_6_6',
 b'10_6_7',
 b'10_6_8',
 b'10_6_9',
 b'10_7_4',
 b'10_7_5',
 b'10_7_6',
 

In [ ]:
strkey = "0_2_4"
ts_dv.read(strkey).result()

Key: b'\x00\x00\x00\x00\x00\x00\x00\x18'


KvStore.ReadResult(state='value', value=b'C\x03\x00\x00\x00\x00\x00\x00\x8b\xaaXD\xc7\x8c1E\xd7\r\x9dE:\x00\x00\x00:\x01\x00\x00\xb8\x01\x00\x00\x00\x00\x02\x00\x02\x00\x02\x00\x13\x00fff\x03\x04^\x99\xff\xbc\x99\xca\xffZf.\x00\xd6\xd4\x00\xd6\xd4\xe4d;\x00\x00F\x94XD\xa0m2E\xae3\x9eE:\x00\x00\x00\x1a\x01\x00\x002\x02\x00\x00\x00\x00\x07\x00L\x00\xd4\x00{\x01fffYJ&\xffs\xcf\xe1s\x00fL.\x00\xd6\xd4\xeb\x00wm\xb3\xff\x00\x00\xc3\x97WD\x19>5E\'\xde\x9eE:\x00\x00\x00:\x01\x00\x00\xb8\x01\x00\x00\x00\x00\x07\x00L\x00\xd4\x00{\x01fffYJ&\xffs\xcf\xe1s\x00fL.\x00\xd6\xd4\x00\xd6\xd4\xe4d;\x00\x00U`YD\x10H1E\xd6\x9c\x9fE:\x00\x00\x00\x1a\x01\x00\x002\x02\x00\x00\x00\x00\x07\x00L\x00\xd4\x00{\x01fffYJ&\xffs\xcf\xe1s\x00fL.\x00\xd6\xd4\xeb\x00wm\xb3\xff\x00\x00N\xa8WDz\x8a4E\x1a\xee\x9dE:\x00\x00\x00:\x01\x00\x00\xb8\x01\x00\x00\x01\x00\x11\x003\x00d\x00\xb3\x00+\x93\xdf\xd0\x00\x00\x7f\x1f\xcc\xffs~.\x99\x96\x00\xd6\xd4\x00\xd6\xd4\xe4d;\x00\x00\x03IWD.M5E\x05\xa9\x9dE:\x00\x00\x00:\x01\x00\x00\

In [25]:
ts_dv.spec()

KvStore.Spec({
  'bucket': 'allen_neuroglancer_ccf',
  'driver': 'gcs',
  'path': 'merscope_ccffix_1/spatial0/',
})

In [3]:
all_ids = pre_ann.get_all_annotation_ids()
all_ids

array([100147785, 100147853, 100147861, ..., 666909423, 671463542,
       671464291], dtype='>u8')

In [4]:
ann=pre_ann.get_by_id(all_ids[0])
ann

{'injection_volume': np.float32(0.1759717),
 'transgenic_line': np.str_('nan'),
 'structure_abbrev': np.str_('PO'),
 'structure_name': np.str_('Posterior complex of the thalamus'),
 'gender': np.str_('M'),
 'structure_color': array([255, 144, 159], dtype=uint8),
 'padding15': array([0], dtype=uint8),
 'structure_id': array([1020], dtype=uint64),
 'experiment_id': array([100147785], dtype=uint64),
 'point': array([7600., 3380., 7070.], dtype=float32)}

In [5]:
df =pre_ann.get_by_relationship('structure_id', ann['structure_id'][0])
df

,geometry,injection_volume,transgenic_line,structure_abbrev,structure_name,gender,structure_color,padding15
ID,,,,,,,,
267999740,"[7540.0, 2400.0, 6990.0]",0.302530,Grik4-Cre,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0]
183011353,"[7340.0, 3550.0, 6870.0]",0.200614,Grik4-Cre,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0]
174781014,"[7440.0, 3600.0, 6920.0]",0.166800,nan,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0]
100147785,"[7600.0, 3380.0, 7070.0]",0.175972,nan,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0]
292212456,"[7560.0, 3450.0, 7110.0]",0.039326,Grik4-Cre,PO,Posterior complex of the thalamus,F,"[255, 144, 159]",[0]
182515576,"[6830.0, 3760.0, 7370.0]",0.045167,Gal-Cre_KI87,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0]
182892855,"[7230.0, 3980.0, 7070.0]",0.025026,Slc18a2-Cre_OZ14,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0]
180708524,"[7350.0, 3930.0, 7040.0]",0.015272,nan,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0]
301674988,"[7430.0, 4110.0, 7440.0]",0.009943,Slc18a2-Cre_OZ14,PO,Posterior complex of the thalamus,F,"[255, 144, 159]",[0]


In [6]:
df = pre_ann.get_all_annotations()
df.head()

,injection_volume,transgenic_line,structure_abbrev,structure_name,gender,structure_color,padding15,structure_id,experiment_id,point
ID,,,,,,,,,,
100147785,0.175972,nan,PO,Posterior complex of the thalamus,M,"[255, 144, 159]",[0],[1020],[100147785],"[7600.0, 3380.0, 7070.0]"
100147853,0.073548,nan,VISp,Primary visual area,M,"[8, 133, 140]",[0],[385],[100147853],"[9440.0, 1220.0, 8630.0]"
100147861,0.138880,nan,CA1,Field CA1,M,"[126, 208, 75]",[0],[382],[100147861],"[6640.0, 1650.0, 7510.0]"
100148142,0.122178,nan,RSPv,"Retrosplenial area, ventral part",M,"[26, 166, 152]",[0],[886],[100148142],"[6830.0, 1260.0, 6160.0]"
100148143,0.114309,nan,DG,Dentate gyrus,M,"[126, 208, 75]",[0],[726],[100148143],"[6690.0, 2690.0, 6190.0]"


In [7]:
pre_ann.get_info()['spatial']

[{'chunk_size': [1024, 1024, 1024],
  'grid_shape': [13, 7, 10],
  'key': 'spatial0',
  'limit': 2994,
  'sharding': {'@type': 'neuroglancer_uint64_sharded_v1',
   'data_encoding': 'gzip',
   'hash': 'murmurhash3_x86_128',
   'minishard_bits': 0,
   'minishard_index_encoding': 'gzip',
   'preshift_bits': 10,
   'shard_bits': 0}}]

In [22]:
pre_ann.ts_by_id.spec().to_json()['metadata']['@type']

'neuroglancer_uint64_sharded_v1'

'neuroglancer_uint64_sharded_v1'